In [1]:
import os
import sys
import keras
#set adequate flag for Theano
import theano
theano.config.gcc.cxxflags = '-march=corei7'
#load needed things
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam, Adagrad, Adadelta, RMSprop
from keras.layers import Input, Activation, Dense
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import backend as K
# Run classifier with cross-validation and plot ROC curves
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy import interp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pyp
import itertools
import math
import ROOT
import cPickle as pickle
from UserFunctions import *

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using Theano backend.


In [2]:
#load events
filein = open('hzz4l_vbf_selection_noDjet_m4l118-130GeV_shuffledFS_JetExtraInfo.pkl','r')
events = pickle.load( filein )
filein.close()

In [3]:
events, scales = prepareJetComponents(events)

In [17]:
#prepare inputs for model
split_factor = 0.8
X_train = []
Y_train = []
sc_train = []
X_test = []
Y_test = []
sc_test = []

for ik in events:
    for iev in range(len(events[ik])):
        if iev < int(split_factor*len(events[ik])):
            for ijet in range(1):#len(events[ik][iev])):
                sc_train.append( scales[ik] )
                X_train.append( events[ik][iev][ijet] )
                if ik == 'VBF':
                    Y_train.append( 1 )
                else:
                    Y_train.append( 0 )
        else:
            for ijet in range(1):#len(events[ik][iev])):
                sc_test.append( scales[ik] )
                X_test.append( events[ik][iev][ijet] )
                if ik == 'VBF':
                    Y_test.append( 1 )
                else:
                    Y_test.append( 0 )

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
sc_train = np.asarray(sc_train)

X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)
sc_test = np.asarray(sc_test)

In [18]:
# updatable plot
from IPython.display import clear_output
class Tscheduler(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.accs = []
        self.val_accs = []
        self.logs = []
        self.trocs = []
        self.vrocs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.accs.append(logs.get('acc'))
        self.val_accs.append(logs.get('val_acc'))
        Y_score = dnn_model.predict(X_train)
        fpr, tpr, thresholds = roc_curve(Y_train, Y_score, sample_weight=sc_train)
        roc_auc = auc(fpr, tpr, reorder=True)
        self.trocs.append(roc_auc)
        Y_score = dnn_model.predict(X_test)
        fpr, tpr, thresholds = roc_curve(Y_test, Y_score, sample_weight=sc_test)
        roc_auc = auc(fpr, tpr, reorder=True)
        self.vrocs.append(roc_auc)
        self.i += 1
        
        clear_output(wait=True)
        fig = pyp.figure()
        fig.set_figheight(6)
        fig.set_figwidth(13)
        
        fig1 = fig.add_subplot(121)
        pyp.plot(self.x, self.losses, label="tloss: %f" % self.losses[len(self.losses)-1]) 
        pyp.plot(self.x, self.val_losses, label="vloss: %f" % self.val_losses[len(self.val_losses)-1])
        #pyp.plot(self.x, self.accs, label="tacc: %f" % self.accs[len(self.accs)-1]) 
        #pyp.plot(self.x, self.val_accs, label="vacc: %f" % self.val_accs[len(self.val_accs)-1])
        #c_lrate = K.eval(self.model.optimizer.lr)
        #pyp.title("Current LR: %f" % c_lrate)
        #c_momentum = K.eval(self.model.optimizer.momentum)
        #pyp.title("Current M: %f" % c_momentum)
        pyp.xlabel('epochs')
        pyp.ylabel('Loss')
        pyp.legend()
        #pyp.ylim([0.7,1.2])
        pyp.grid(True)
        #pyp.show()
        
        fig2 = fig.add_subplot(122)
        pyp.plot(self.x, self.trocs, color='blue', label='Current train AUC = %0.3f' % self.trocs[len(self.trocs)-1])
        pyp.plot(self.x, self.vrocs, color='red', label='Current   test AUC = %0.3f' % self.vrocs[len(self.vrocs)-1])
        #pyp.ylim([0, 1.0])
        pyp.xlabel('epochs')
        pyp.ylabel('AUC')
        pyp.title('ROC evolution')
        pyp.legend()
        pyp.grid(True)
        #pyp.show()

        pyp.tight_layout()
        fig = pyp.show()

        #changing lr based on loss
        #if(len(self.losses) > 20):
        #    lsum1 = 0
        #    lsum2 = 0
        #    for i in range(6):
        #        if(i < 3):
        #            lsum1 += self.losses[len(self.losses)-1-i]/3.
        #        else:
        #            lsum2 += self.losses[len(self.losses)-1-i]/3.
        #    if(math.fabs(lsum2-lsum1) < 0.01):
        #        n_lrate = c_lrate*(1 - 1/20.)
        #        K.set_value(self.model.optimizer.lr, n_lrate)
        #else:
        #    n_lrate = self.losses[len(self.losses)-1]*0.001
        #    K.set_value(self.model.optimizer.lr, n_lrate)
        
        #initial_lrate = 0.001
        #drop = 0.6
        #epochs_drop = 100
        #lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
        #K.set_value(self.model.optimizer.lr, 0.001)
        
        #stop training if training and testing Loss get too away
        #if(epoch > 50):
        #    train_sum = 0
        #    val_sum = 0
        #    for i in range(30):
        #        train_sum += self.losses[len(self.losses)-1-i]/6.
        #        val_sum += self.val_losses[len(self.val_losses)-1-i]/6.
        #    if(train_sum-val_sum < -0.02):
        #        self.model.stop_training = True
        #        print "Stoping training!"
                
            
Tschedule = Tscheduler()

In [25]:
#parameters for training
nepochs = 20
wait_for = 60
sbatch = 128
opt = Adam()

early_stopping = EarlyStopping(monitor='val_loss', patience=wait_for)

In [28]:
#DNN network
ninputs = len(X_train[0])
print "---------- DNN Topology --------- "
print "Inputs: %i" % ninputs

dnn_model = Sequential()
dnn_model.add(Dense(100, input_shape=(ninputs,), activation='relu', kernel_initializer='random_uniform'))
dnn_model.add(Dense(60, activation='relu', kernel_initializer='random_uniform'))    
dnn_model.add(Dense(30, activation='relu', kernel_initializer='random_uniform'))    
dnn_model.add(Dense(1, activation='sigmoid', kernel_initializer='random_uniform'))    
dnn_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print dnn_model.summary()

---------- DNN Topology --------- 
Inputs: 300
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_30 (Dense)             (None, 60)                6060      
_________________________________________________________________
dense_31 (Dense)             (None, 30)                1830      
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 31        
Total params: 38,021
Trainable params: 38,021
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
#filepath1="best_weights.hdf5"
#checkpoint = ModelCheckpoint(filepath1, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

#train the DNN
history1 = dnn_model.fit(X_train,
                        Y_train, 
                        sample_weight=sc_train,
                        validation_data=(X_test,
                                         Y_test
                                         ,sc_test
                                        ), 
                        epochs=nepochs,
                        batch_size=sbatch, 
                        verbose=2,
                        callbacks=[early_stopping]#,Tschedule]
                       )

Train on 63828 samples, validate on 15959 samples
Epoch 1/20
2s - loss: 1.3700 - acc: 0.5508 - val_loss: 1.3659 - val_acc: 0.5808
Epoch 2/20
3s - loss: 1.3370 - acc: 0.5738 - val_loss: 1.3429 - val_acc: 0.5780
Epoch 3/20
3s - loss: 1.3279 - acc: 0.5849 - val_loss: 1.3367 - val_acc: 0.5906
Epoch 4/20
3s - loss: 1.3229 - acc: 0.5874 - val_loss: 1.3360 - val_acc: 0.5995
Epoch 5/20
3s - loss: 1.3178 - acc: 0.5912 - val_loss: 1.3408 - val_acc: 0.6025
Epoch 6/20
3s - loss: 1.3112 - acc: 0.5957 - val_loss: 1.3329 - val_acc: 0.6013
Epoch 7/20
4s - loss: 1.3096 - acc: 0.5959 - val_loss: 1.3276 - val_acc: 0.5950
Epoch 8/20
4s - loss: 1.3055 - acc: 0.5963 - val_loss: 1.3427 - val_acc: 0.5943
Epoch 9/20
4s - loss: 1.3020 - acc: 0.5973 - val_loss: 1.3282 - val_acc: 0.5852
Epoch 10/20
4s - loss: 1.2988 - acc: 0.5985 - val_loss: 1.3328 - val_acc: 0.6010
Epoch 11/20
4s - loss: 1.2964 - acc: 0.5985 - val_loss: 1.3356 - val_acc: 0.5910
Epoch 12/20
4s - loss: 1.2933 - acc: 0.6020 - val_loss: 1.3306 - val